# Gerando dados csv: Análise estatísica de diferentes ruídos utilizando teste de Friedman

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import glob
from utils.experiments_functions import get_img_info_simple
import utils.noise_generator as noise_gen
import utils.utils as utils_functions
from skimage.restoration import (
    denoise_tv_chambolle,
)

In [17]:
def generate_noisy_images(original_image, noise_levels):
    """
    Generate images with different types of noise at various levels.
    """
    noisy_images = {}
    
    for noise_type in ['gaussian', 'salt_pepper', 'speckle', 'poisson']:
        noisy_images[noise_type] = []
        for level in noise_levels:
            if noise_type == 'salt_pepper':
                noisy_img = noise_gen.add_sp_noise_0to1(original_image, intensity=level)
            elif noise_type == 'gaussian':
                noisy_img = noise_gen.add_gaussian_noise_0to1(original_image, intensity=level)
            elif noise_type == 'speckle':
                noisy_img = noise_gen.add_speckle_noise_0to1(original_image, intensity=level)
            elif noise_type == 'poisson':
                noisy_img = noise_gen.add_poisson_noise_0to1(original_image, intensity=level)
            noisy_images[noise_type].append(noisy_img)
    
    return noisy_images


def add_noise_generic(original_image, noise_type, noise_level):
    """
    Add given noise
    """    
    if noise_type not in ['gaussian', 'salt_and_pepper', 'speckle', 'poisson']:
        raise ValueError("noise_type should be in ['gaussian', 'salt_and_pepper', 'speckle', 'poisson']")
        
    if noise_type == 'salt_and_pepper':
        noisy_img = noise_gen.add_sp_noise_0to1(original_image, intensity=noise_level)
    elif noise_type == 'gaussian':
        noisy_img = noise_gen.add_gaussian_noise_0to1(original_image, intensity=noise_level)
    elif noise_type == 'speckle':
        noisy_img = noise_gen.add_speckle_noise_0to1(original_image, intensity=noise_level)
    elif noise_type == 'poisson':
        noisy_img = noise_gen.add_poisson_noise_0to1(original_image, intensity=noise_level)
        
    return noisy_img


## Uma imagem de cada classe SIntorn

In [18]:
# Reads 1 image from each class
images_path = f'data/Sintorn/hardware'
rot = 'r000'
seg = 's001'
images_filenames = []
for file in os.listdir(images_path):
    if file.endswith(f"{rot}-{seg}.png"):
        images_filenames.append(file)
print(len(images_filenames))
print(images_filenames)

25
['canesugar01-r000-s001.png', 'canvas01-r000-s001.png', 'couscous01-r000-s001.png', 'fabric01-r000-s001.png', 'fabric02-r000-s001.png', 'fabric03-r000-s001.png', 'fabric04-r000-s001.png', 'fabric05-r000-s001.png', 'flaxseed01-r000-s001.png', 'knitwear01-r000-s001.png', 'knitwear02-r000-s001.png', 'knitwear03-r000-s001.png', 'lentils01-r000-s001.png', 'oatmeal01-r000-s001.png', 'pearlsugar01-r000-s001.png', 'rice01-r000-s001.png', 'rug01-r000-s001.png', 'ryeflakes01-r000-s001.png', 'seeds01-r000-s001.png', 'seeds02-r000-s001.png', 'sprinkles01-r000-s001.png', 'tile01-r000-s001.png', 'towel01-r000-s001.png', 'wheat01-r000-s001.png', 'wool01-r000-s001.png']


In [19]:
# Sintorn images
all_images = {}

# Display the images
for i, filename in enumerate(images_filenames):
    print(f'Processing image {i+1}/{len(images_filenames)}: {filename}')
    # Reads image
    gs_image = cv2.imread(f'{images_path}/{filename}', cv2.IMREAD_GRAYSCALE)
    gs_image = utils_functions.normalize_uint8_to_float64(gs_image)
    
    # Adds image to the all_images dictionary
    item = filename.split('.')[0]
    if item not in all_images:
        all_images[item] = gs_image

Processing image 1/25: canesugar01-r000-s001.png
Processing image 2/25: canvas01-r000-s001.png
Processing image 3/25: couscous01-r000-s001.png
Processing image 4/25: fabric01-r000-s001.png
Processing image 5/25: fabric02-r000-s001.png
Processing image 6/25: fabric03-r000-s001.png
Processing image 7/25: fabric04-r000-s001.png
Processing image 8/25: fabric05-r000-s001.png
Processing image 9/25: flaxseed01-r000-s001.png
Processing image 10/25: knitwear01-r000-s001.png
Processing image 11/25: knitwear02-r000-s001.png
Processing image 12/25: knitwear03-r000-s001.png
Processing image 13/25: lentils01-r000-s001.png
Processing image 14/25: oatmeal01-r000-s001.png
Processing image 15/25: pearlsugar01-r000-s001.png
Processing image 16/25: rice01-r000-s001.png
Processing image 17/25: rug01-r000-s001.png
Processing image 18/25: ryeflakes01-r000-s001.png
Processing image 19/25: seeds01-r000-s001.png
Processing image 20/25: seeds02-r000-s001.png
Processing image 21/25: sprinkles01-r000-s001.png
Proc

In [20]:
# Process all images
noise_types = ['original', 'salt_and_pepper', 'speckle', 'gaussian', 'poisson']

# set noise intensities
noise_levels = [0.01, 0.05,  0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]

image_data_list = []
for i, item in enumerate(all_images):
    print(f"Processing image {i+1}/{len(all_images)}: {item}")
    # get image
    gs_image = all_images[item]
    img_filename = item
    # extract metrics
    print("\tExtracting data")
    image_data = get_img_info_simple(gs_image)
    # add the information
    image_data['image_name'] = img_filename
    image_data['noise_type'] = 'original'
    image_data['noise_param'] = 0
    image_data['is_iso_noise'] = False
    image_data_list.append(image_data)
    image_data = image_data.copy()
    image_data['is_iso_noise'] = True
    image_data_list.append(image_data)
    # adds no noise information to each noise_type
    for n_type in noise_types:
        if n_type != 'original':
            # adds the information
            image_data = image_data.copy()
            image_data['is_iso_noise'] = False
            image_data['noise_type'] = n_type
            image_data_list.append(image_data)
            image_data = image_data.copy()
            image_data['is_iso_noise'] = True
            image_data_list.append(image_data)

    # add noises
    print("\tAdding noise")
    for prob in noise_levels:
        # salt and pepper
        print("\t\tAdding S&P noise")
        noised_image = add_noise_generic(gs_image, 'salt_and_pepper', prob)
        
        # extract metrics
        print("\t\tExtracting data - Noised")
        noised_image_data = get_img_info_simple(noised_image)
        # add the information
        noised_image_data['image_name'] = img_filename
        noised_image_data['noise_type'] = 'salt_and_pepper'
        noised_image_data['noise_param'] = prob
        noised_image_data['is_iso_noise'] = False
        image_data_list.append(noised_image_data)
        
        # isolated noise        
        denoised_image = denoise_tv_chambolle(noised_image, weight=0.1, channel_axis=None)
        subtracted_image = utils_functions.normalize_outbound_to_float64(noised_image - denoised_image)        
        # extract metrics
        print("\t\tExtracting data - Isolated noise")
        noised_image_data = get_img_info_simple(subtracted_image)
        # add the information
        noised_image_data['image_name'] = img_filename
        noised_image_data['noise_type'] = 'salt_and_pepper'
        noised_image_data['noise_param'] = prob
        noised_image_data['is_iso_noise'] = True
        image_data_list.append(noised_image_data)
        
        
        # gaussian
        print("\t\tAdding Gaussian noise")
        noised_image = add_noise_generic(gs_image, 'gaussian', prob)
        
        # extract metrics
        print("\t\tExtracting data - Noised")
        noised_image_data = get_img_info_simple(noised_image)
        # add the information
        noised_image_data['image_name'] = img_filename
        noised_image_data['noise_type'] = 'gaussian'
        noised_image_data['noise_param'] = prob
        noised_image_data['is_iso_noise'] = False
        image_data_list.append(noised_image_data)
        
        # isolated noise        
        denoised_image = denoise_tv_chambolle(noised_image, weight=0.1, channel_axis=None)
        subtracted_image = utils_functions.normalize_outbound_to_float64(noised_image - denoised_image)        
        # extract metrics
        print("\t\tExtracting data - Isolated noise")
        noised_image_data = get_img_info_simple(subtracted_image)
        # add the information
        noised_image_data['image_name'] = img_filename
        noised_image_data['noise_type'] = 'gaussian'
        noised_image_data['noise_param'] = prob
        noised_image_data['is_iso_noise'] = True
        image_data_list.append(noised_image_data)
        
        
        # speckle
        print("\t\tAdding Speckle noise")
        noised_image = add_noise_generic(gs_image, 'speckle', prob)
        
        # extract metrics
        print("\t\tExtracting data - Noised")
        noised_image_data = get_img_info_simple(subtracted_image)
        # add the information
        noised_image_data['image_name'] = img_filename
        noised_image_data['noise_type'] = 'speckle'
        noised_image_data['noise_param'] = prob
        noised_image_data['is_iso_noise'] = False
        image_data_list.append(noised_image_data)
        
        # isolated noise        
        denoised_image = denoise_tv_chambolle(noised_image, weight=0.1, channel_axis=None)
        subtracted_image = utils_functions.normalize_outbound_to_float64(noised_image - denoised_image)        
        # extract metrics
        print("\t\tExtracting data - Isolated noise")
        noised_image_data = get_img_info_simple(subtracted_image)
        # add the information
        noised_image_data['image_name'] = img_filename
        noised_image_data['noise_type'] = 'speckle'
        noised_image_data['noise_param'] = prob
        noised_image_data['is_iso_noise'] = True
        image_data_list.append(noised_image_data)
        
        
        # poisson
        print("\t\tAdding Poisson noise")
        noised_image = add_noise_generic(gs_image, 'poisson', prob)
        
        # extract metrics
        print("\t\tExtracting data - Noised")
        noised_image_data = get_img_info_simple(subtracted_image)
        # add the information
        noised_image_data['image_name'] = img_filename
        noised_image_data['noise_type'] = 'poisson'
        noised_image_data['noise_param'] = prob
        noised_image_data['is_iso_noise'] = False
        image_data_list.append(noised_image_data)
        
        # isolated noise        
        denoised_image = denoise_tv_chambolle(noised_image, weight=0.1, channel_axis=None)
        subtracted_image = utils_functions.normalize_outbound_to_float64(noised_image - denoised_image)        
        # extract metrics
        print("\t\tExtracting data - Isolated noise")
        noised_image_data = get_img_info_simple(subtracted_image)
        # add the information
        noised_image_data['image_name'] = img_filename
        noised_image_data['noise_type'] = 'poisson'
        noised_image_data['noise_param'] = prob
        noised_image_data['is_iso_noise'] = True
        image_data_list.append(noised_image_data)

Processing image 1/25: canesugar01-r000-s001
	Extracting data
	Adding noise
		Adding S&P noise
		Extracting data - Noised
		Extracting data - Isolated noise
		Adding Gaussian noise
		Extracting data - Noised
		Extracting data - Isolated noise
		Adding Speckle noise
		Extracting data - Noised
		Extracting data - Isolated noise
		Adding Poisson noise
		Extracting data - Noised
		Extracting data - Isolated noise
		Adding S&P noise
		Extracting data - Noised
		Extracting data - Isolated noise
		Adding Gaussian noise
		Extracting data - Noised
		Extracting data - Isolated noise
		Adding Speckle noise
		Extracting data - Noised
		Extracting data - Isolated noise
		Adding Poisson noise
		Extracting data - Noised
		Extracting data - Isolated noise
		Adding S&P noise
		Extracting data - Noised
		Extracting data - Isolated noise
		Adding Gaussian noise
		Extracting data - Noised
		Extracting data - Isolated noise
		Adding Speckle noise
		Extracting data - Noised
		Extracting data - Isolated nois

In [21]:
data_default_aux = pd.concat(image_data_list, ignore_index=True)
data_default_aux.to_csv(f"results/qualificacao_noises/statistical_analysis/sintorn_noises.csv", index=False)